<a href="https://colab.research.google.com/github/Yazhinimaran-EM/NASA_NEAR_EARTH_OBJECTS-DATA-TRACKING-INSIGHTS/blob/main/NASA_NEAR_EARTH_OBJECTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
API_KEY= 'QJbMRtL5ujDIP63Zma5PMsQ7XL7CuSxAFOPoudY7'
import requests
url=f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_KEY}"

requests.get(url)
target = 10000

In [7]:
# Step 1: Fetch data until we reach 10000 unique asteroids
import requests
from collections import defaultdict
from datetime import datetime
unique_asteroids = {}
nasa_data = []


while len(unique_asteroids) < target:
    print(f"Collected {len(unique_asteroids)} unique asteroids... Fetching from: {url}")
    response = requests.get(url)
    data = response.json()
    details = data['near_earth_objects']

    for date, ast_details in details.items():
        for ast in ast_details:
            if 'close_approach_data' not in ast or not ast['close_approach_data']:
                continue

            approach = ast['close_approach_data'][0]

            asteroid_data = {
                'id': int(ast['id']),
                'neo_reference_id': ast['neo_reference_id'],
                'name': ast['name'],
                'absolute_magnitude_h': ast['absolute_magnitude_h'],
                'estimated_diameter_min_km': ast['estimated_diameter']['kilometers']['estimated_diameter_min'],
                'estimated_diameter_max_km': ast['estimated_diameter']['kilometers']['estimated_diameter_max'],
                'is_potentially_hazardous_asteroid': ast['is_potentially_hazardous_asteroid'],
                'close_approach_data': datetime.strptime(approach['close_approach_date'], "%Y-%m-%d").date(),
                'relative_velocity_km': float(approach['relative_velocity']['kilometers_per_hour']),
                'astronomical_unit': float(approach['miss_distance']['astronomical']),
                'miss_distance_kilometers': float(approach['miss_distance']['kilometers']),
                'miss_distance_lunar': float(approach['miss_distance']['lunar']),
                'orbiting_body': approach['orbiting_body']
            }

            nasa_data.append(asteroid_data)

            # Store only unique asteroids by ID
            if asteroid_data['id'] not in unique_asteroids:
                unique_asteroids[asteroid_data['id']] = asteroid_data

            if len(unique_asteroids) >= target:
                break
        if len(unique_asteroids) >= target:
            break

    url = data['links']['next']  # Move to next page

Collected 0 unique asteroids... Fetching from: https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key=QJbMRtL5ujDIP63Zma5PMsQ7XL7CuSxAFOPoudY7
Collected 129 unique asteroids... Fetching from: http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-08&end_date=2024-01-15&detailed=false&api_key=QJbMRtL5ujDIP63Zma5PMsQ7XL7CuSxAFOPoudY7
Collected 268 unique asteroids... Fetching from: http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-15&end_date=2024-01-22&detailed=false&api_key=QJbMRtL5ujDIP63Zma5PMsQ7XL7CuSxAFOPoudY7
Collected 383 unique asteroids... Fetching from: http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-22&end_date=2024-01-29&detailed=false&api_key=QJbMRtL5ujDIP63Zma5PMsQ7XL7CuSxAFOPoudY7
Collected 522 unique asteroids... Fetching from: http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-29&end_date=2024-02-05&detailed=false&api_key=QJbMRtL5ujDIP63Zma5PMsQ7XL7CuSxAFOPoudY7
Collected 672 unique asteroids... Fetching from: http://

In [8]:
len(nasa_data)

14796

In [9]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 55.2 MB/s eta 0:00:00


In [10]:
import requests
import json
from datetime import datetime
import mysql.connector

# Step 2: Connect to MySQL
conn = mysql.connector.connect(
    host="gateway01.eu-central-1.prod.aws.tidbcloud.com",
    user="Tf6RqboHNPV8kyH.root",
    password="fn8OtaiwT8YNm99R",
    port=4000,
    database="Nasa"
)
cursor = conn.cursor()

# Step 3: Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS Asteroids (
    id INT PRIMARY KEY,
    name VARCHAR(100),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS close_approach (
    id INT AUTO_INCREMENT PRIMARY KEY,
    neo_reference_id VARCHAR(50),
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(100)
)
""")

conn.commit()

# Step 4: Insert unique asteroids
unique_asteroids = {}
for i in nasa_data:
    if i['id'] not in unique_asteroids:
        unique_asteroids[i['id']] = i

insert_asteroids_sql = """
INSERT INTO Asteroids (
    id,
    name,
    absolute_magnitude_h,
    estimated_diameter_min_km,
    estimated_diameter_max_km,
    is_potentially_hazardous_asteroid
) VALUES (%s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    name=VALUES(name),
    absolute_magnitude_h=VALUES(absolute_magnitude_h),
    estimated_diameter_min_km=VALUES(estimated_diameter_min_km),
    estimated_diameter_max_km=VALUES(estimated_diameter_max_km),
    is_potentially_hazardous_asteroid=VALUES(is_potentially_hazardous_asteroid)

"""

for asteroid in unique_asteroids.values():
    cursor.execute(insert_asteroids_sql, (
        asteroid['id'],
        asteroid['name'],
        asteroid['absolute_magnitude_h'],
        asteroid['estimated_diameter_min_km'],
        asteroid['estimated_diameter_max_km'],
        asteroid['is_potentially_hazardous_asteroid']
    ))

conn.commit()

# Step 5: Insert close approach data
insert_close_approach_sql = """
INSERT INTO close_approach (
    neo_reference_id,
    close_approach_date,
    relative_velocity_kmph,
    astronomical,
    miss_distance_km,
    miss_distance_lunar,
    orbiting_body
) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for i in nasa_data:
    cursor.execute(insert_close_approach_sql, (
        i['neo_reference_id'],
        i['close_approach_data'],
        i['relative_velocity'],
        i['astronomical_unit'],
        i['miss_distance_kilometers'],
        i['miss_distance_lunar'],
        i['orbiting_body']
    ))

conn.commit()
cursor.close()
conn.close()

print("✅ Successfully inserted unique asteroids and close approach data.")

InterfaceError: 2013: Lost connection to MySQL server during query

In [ ]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(host="gateway01.eu-central-1.prod.aws.tidbcloud.com",
                               user="Tf6RqboHNPV8kyH.root",
                               password="fn8OtaiwT8YNm99R",
                               port = 4000,
                               database = "Nasa") # Connect to the 'Nasa' database
cursor = conn.cursor() # Re-create the cursor after connecting to the database

cursor.execute("SELECT * FROM asteroids")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]

# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns=columns)
display(new_df) # shows all the values inserted - 7 columns

cursor.close()
conn.close()

,id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid
0,2000887,887 Alinda (A918 AA),13.820,4.576726,10.233872,0
1,2001685,1685 Toro (1948 OA),14.280,3.703011,8.280184,0
2,2002063,2063 Bacchus (1977 HB),17.210,0.960627,2.148028,0
3,2002340,2340 Hathor (1976 UA),20.430,0.218049,0.487573,1
4,2004034,4034 Vishnu (1986 PA),18.490,0.532789,1.191352,1
...,...,...,...,...,...,...
7498,54537654,(2025 MB90),24.140,0.039496,0.088316,0
7499,54537919,(2025 NL),27.558,0.008184,0.018300,0
7500,54537956,(2025 NT),25.310,0.023044,0.051528,0
7501,54537957,(2025 NU),25.101,0.025372,0.056734,0


In [ ]:
import pandas as pd
from datetime import datetime
import mysql.connector

In [ ]:
conn = mysql.connector.connect(host="gateway01.eu-central-1.prod.aws.tidbcloud.com",
                               user="Tf6RqboHNPV8kyH.root",
                               password="fn8OtaiwT8YNm99R",
                               port = 4000,
                               database = "Nasa") # Connect to the 'Nasa' database
cursor = conn.cursor() # Re-create the cursor after connecting to the database

In [ ]:
unique_asteroids = {}
for i in nasa_data:
    if i['id'] not in unique_asteroids:
        unique_asteroids[i['id']] = i

In [ ]:
df = pd.DataFrame(nasa_data)

In [ ]:
df.columns

Index(['id', 'neo_reference_id', 'name', 'absolute_magnitude_h',
       'estimated_diameter_min_km', 'estimated_diameter_max_km',
       'is_potentially_hazardous_asteroid', 'close_approach_data',
       'relative_velocity', 'astronomical_unit', 'miss_distance_kilometers',
       'miss_distance_lunar', 'orbiting_body'],
      dtype='object')

In [ ]:
def show_query(query):
    df = pd.read_sql(query, con=conn)  # Use pandas read_sql
    display(df)

In [ ]:
query1 = '''
SELECT neo_reference_id, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id
ORDER BY approach_count DESC
'''
show_query(query1)

/tmp/ipython-input-39-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,neo_reference_id,approach_count
0,54471061,17
1,3825135,17
2,54342587,17
3,54526031,17
4,54293876,17
...,...,...
7496,54122200,3
7497,54468793,3
7498,3989396,3
7499,54520320,3


In [ ]:
query2 = '''
SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY avg_velocity DESC
'''
show_query(query2)


/tmp/ipython-input-39-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,neo_reference_id,avg_velocity
0,54277079,190513.421875
1,54392072,173071.828125
2,3803884,161520.890625
3,3790060,158279.859375
4,3425295,154336.406250
...,...,...
7496,54471067,2839.155451
7497,54459240,2149.995361
7498,3798020,1909.578125
7499,54440316,1610.566895


In [ ]:
query3 = '''
SELECT neo_reference_id, MAX(relative_velocity_kmph) AS max_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY max_velocity DESC
LIMIT 10
'''
show_query(query3)

/tmp/ipython-input-39-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,neo_reference_id,max_velocity
0,54277079,190513.42
1,54392072,173071.83
2,3803884,161520.89
3,3790060,158279.86
4,3425295,154336.40
5,3092220,152218.22
6,3588900,151520.39
7,54448599,150599.45
8,54111253,150583.02
9,3383901,149261.28


In [ ]:
query4 = '''
SELECT ca.neo_reference_id, COUNT(*) AS approach_count
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE a.is_potentially_hazardous_asteroid = 1
GROUP BY ca.neo_reference_id
HAVING COUNT(*) > 3
'''
show_query(query4)

/tmp/ipython-input-39-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,neo_reference_id,approach_count
0,54494098,18
1,2779817,60
2,3781344,24
3,2756327,18
4,2503941,18
...,...,...
385,3616698,24
386,3581669,60
387,3293926,80
388,3373682,24


In [ ]:
query5 = '''
SELECT DATE_FORMAT(close_approach_date, '%Y-%m') AS month, COUNT(*) AS count
FROM close_approach
GROUP BY month
ORDER BY count DESC
LIMIT 1
'''
show_query(query5)

/tmp/ipython-input-18-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,month,count
0,2024-04,2848


In [ ]:
query6 = '''
SELECT neo_reference_id, relative_velocity_kmph AS fastest_speed
FROM close_approach
ORDER BY relative_velocity_kmph DESC
LIMIT 1
'''
show_query(query6)

/tmp/ipython-input-18-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,neo_reference_id,fastest_speed
0,54277079,190513.42


In [ ]:
conn = mysql.connector.connect(host="gateway01.eu-central-1.prod.aws.tidbcloud.com",
                               user="Tf6RqboHNPV8kyH.root",
                               password="fn8OtaiwT8YNm99R",
                               port = 4000,
                               database = "Nasa") # Connect to the 'Nasa' database
cursor = conn.cursor() # Re-create the cursor after connecting to the database

In [ ]:
def show_query(query):
    df = pd.read_sql(query, con=conn)  # Use pandas read_sql
    display(df)

In [ ]:
query7 = '''
SELECT id, name, estimated_diameter_max_km
FROM asteroids
ORDER BY estimated_diameter_max_km DESC
'''
show_query(query7)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,id,name,estimated_diameter_max_km
0,2000887,887 Alinda (A918 AA),10.233872
1,2001685,1685 Toro (1948 OA),8.280184
2,2066146,66146 (1998 TU3),7.980681
3,2005660,5660 (1974 MA),4.989304
4,2533671,533671 (2014 LJ21),3.926811
...,...,...,...
7498,3556206,(2011 CQ1),0.002260
7499,54511953,(2024 YW8),0.002208
7500,54380194,(2023 RS),0.002042
7501,54473959,(2024 RC42),0.001986


In [ ]:
query8 = '''
SELECT *
FROM close_approach
ORDER BY neo_reference_id, close_approach_date
'''
show_query(query8)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,id,neo_reference_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,8021,2000887,2025-01-08,29695.316,0.082198,12296605.0,31.974916,Earth
1,376,2001685,2024-01-20,58226.723,0.133063,19905934.0,51.761490,Earth
2,1864,2002063,2024-03-31,30698.844,0.120005,17952526.0,46.682030,Earth
3,6504,2002340,2024-11-02,93669.380,0.397608,59481376.0,154.669680,Earth
4,9574,2004034,2025-04-01,43128.810,0.156758,23450596.0,60.978690,Earth
...,...,...,...,...,...,...,...,...
9995,3961,54537654,2024-07-17,28745.035,0.134635,20141160.0,52.373146,Earth
9996,3767,54537919,2024-07-04,24871.105,0.150594,22528596.0,58.581207,Earth
9997,3863,54537956,2024-07-10,34355.215,0.103052,15416412.0,40.087364,Earth
9998,3243,54537957,2024-06-08,37724.670,0.165831,24807972.0,64.508280,Earth


In [ ]:
query9 = '''
SELECT a.name, MIN(ca.miss_distance_km) AS closest_approach
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
GROUP BY a.id
ORDER BY closest_approach ASC
'''
show_query(query9)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,name,closest_approach
0,(2024 XA),7.726027e+03
1,(2024 LH1),8.098256e+03
2,(2024 UG9),8.849866e+03
3,(2025 BP6),9.711953e+03
4,(2024 HA),1.555513e+04
...,...,...
7498,(2019 LX),7.476754e+07
7499,(2020 YH),7.477857e+07
7500,(2023 GF),7.478050e+07
7501,(2021 UQ3),7.478990e+07


In [ ]:
query10 = '''
SELECT DISTINCT a.name, ca.relative_velocity_kmph
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.relative_velocity_kmph > 50000
'''
show_query(query10)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,name,relative_velocity_kmph
0,(2024 GW2),69015.210
1,(2020 TK4),55806.227
2,613286 (2005 YQ96),56413.043
3,(2022 WQ3),69606.180
4,(2021 RL7),61494.190
...,...,...
3107,(2014 YC),56895.690
3108,(2024 UA10),55703.633
3109,(2024 ST1),59018.965
3110,(2024 JL8),56026.560


In [ ]:
query11 = '''
SELECT DATE_FORMAT(close_approach_date, '%Y-%m') AS month, COUNT(*) AS total
FROM close_approach
GROUP BY month
ORDER BY total DESC
'''
show_query(query11)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,month,total
0,2024-10,908
1,2024-09,841
2,2024-04,712
3,2024-12,693
4,2024-11,674
5,2024-03,637
6,2024-01,628
7,2025-03,622
8,2024-02,621
9,2024-08,591


In [ ]:
query12 = '''
SELECT id, name, absolute_magnitude_h
FROM asteroids
ORDER BY absolute_magnitude_h ASC
LIMIT 1
'''
show_query(query12)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,id,name,absolute_magnitude_h
0,2000887,887 Alinda (A918 AA),13.82


In [ ]:
query13 = '''
SELECT is_potentially_hazardous_asteroid, COUNT(*) AS count
FROM asteroids
GROUP BY is_potentially_hazardous_asteroid
'''
show_query(query13)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,is_potentially_hazardous_asteroid,count
0,1,390
1,0,7113


In [ ]:
query14 = '''
SELECT a.name, ca.close_approach_date, ca.miss_distance_lunar
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.miss_distance_lunar < 1
ORDER BY ca.miss_distance_lunar
'''
show_query(query14)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,name,close_approach_date,miss_distance_lunar
0,(2024 XA),2024-12-01,0.020090
1,(2024 LH1),2024-06-06,0.021058
2,(2024 UG9),2024-10-30,0.023012
3,(2025 BP6),2025-01-26,0.025254
4,(2024 HA),2024-04-16,0.040448
...,...,...,...
231,(2024 UB1),2024-10-18,0.968912
232,(2024 JL3),2024-05-08,0.979100
233,(2024 RR8),2024-09-02,0.979981
234,(2024 RR8),2024-09-02,0.979981


In [ ]:
query15 = '''
SELECT a.name, ca.close_approach_date, ca.astronomical
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.astronomical < 0.05
ORDER BY ca.astronomical
'''
show_query(query15)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,name,close_approach_date,astronomical
0,(2024 XA),2024-12-01,0.000052
1,(2024 LH1),2024-06-06,0.000054
2,(2024 UG9),2024-10-30,0.000059
3,(2025 BP6),2025-01-26,0.000065
4,(2024 HA),2024-04-16,0.000104
...,...,...,...
2535,(2024 UF2),2024-11-04,0.049892
2536,(2024 QJ1),2024-07-27,0.049906
2537,(2006 GB1),2024-10-08,0.049947
2538,(2024 FV),2024-03-16,0.049956


ADDTIONAL QUERIES

In [ ]:
# 💡 ADD 1: Most frequently orbiting body (other than Earth)
Add_1 = ''' SELECT orbiting_body, COUNT(*) AS count
FROM close_approach
WHERE orbiting_body != 'Earth'
GROUP BY orbiting_body
ORDER BY count DESC '''
show_query(Add_1)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,orbiting_body,count


In [ ]:
# 💡 Additional Query 2: Average miss distance by hazard type
Add_2 = '''
SELECT a.is_potentially_hazardous_asteroid, AVG(ca.miss_distance_km) AS avg_miss_distance
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
GROUP BY a.is_potentially_hazardous_asteroid
'''
show_query(Add_2)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,is_potentially_hazardous_asteroid,avg_miss_distance
0,0,2.846930e+07
1,1,3.531738e+07


In [ ]:
# 💡 Additional Query 3: Top 5 closest approaches to Earth
Add_3 = '''
SELECT a.name, ca.close_approach_date, ca.miss_distance_km
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
ORDER BY ca.miss_distance_km ASC
LIMIT 5
'''
show_query(Add_3)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,name,close_approach_date,miss_distance_km
0,(2024 XA),2024-12-01,7726.0270
1,(2024 LH1),2024-06-06,8098.2563
2,(2024 UG9),2024-10-30,8849.8660
3,(2025 BP6),2025-01-26,9711.9530
4,(2024 HA),2024-04-16,15555.1270


In [ ]:
# Additional Query 4: Total unique hazardous asteroids
Add_4 = '''
SELECT COUNT(DISTINCT id) AS hazardous_asteroid_count
FROM asteroids
WHERE is_potentially_hazardous_asteroid = 1
'''
show_query(Add_4)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,hazardous_asteroid_count
0,390


In [ ]:
# 💡 Additional Query 5: Asteroids that passed within 1 LD more than once
Add_5 = '''
SELECT ca.neo_reference_id, a.name, COUNT(*) AS close_pass_count
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.miss_distance_lunar < 1
GROUP BY ca.neo_reference_id
HAVING COUNT(*) > 1
ORDER BY close_pass_count DESC
'''
show_query(Add_5)

/tmp/ipython-input-5-2685788363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)  # Use pandas read_sql


,neo_reference_id,name,close_pass_count
0,54511953,(2024 YW8),2
1,54524319,(2025 FK3),2
2,54499685,(2024 VP4),2
3,54434347,(2024 GZ5),2
4,54487958,(2024 TR4),2
5,54429348,(2024 EF),2
6,54426745,(2024 CM5),2
7,54480957,(2024 ST),2
8,54520710,(2025 DT5),2
9,54502879,(2024 XH1),2


In [ ]:
!pip install streamlit streamlit_option_menu   # installing streamlit and streamlit_option_menu packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 39.3 MB/s eta 0:00:00


In [ ]:
%%writefile Nasa.py
# NASA Asteroid Tracker App using Streamlit

import streamlit as st
import pandas as pd
from datetime import datetime
import mysql.connector
from streamlit_option_menu import option_menu

# MySQL connection setup
conn = mysql.connector.connect(
    host="gateway01.eu-central-1.prod.aws.tidbcloud.com",
    user="Tf6RqboHNPV8kyH.root",
    password="fn8OtaiwT8YNm99R",
    port=4000,
    database="Nasa"
)
cursor = conn.cursor()

# Page config
st.set_page_config(layout="wide")
st.markdown("<h1 style='text-align: center; color: #4B8BBE;'>🚀 NASA Asteroid Tracker 🌠</h1>", unsafe_allow_html=True)
st.divider()

# Sidebar menu
with st.sidebar:
    selected = option_menu(
        menu_title="Asteroid Approaches",
        options=["Filter Criteria", "Queries"],
        icons=["filter", "list"],
        menu_icon="cast",
        default_index=0
    )

# --- Filter Section ---
if selected == "Filter Criteria":
    c1, a, c2, b, c3 = st.columns([0.2, 0.1, 0.2, 0.1, 0.2])

    with c1:
        mag_min = st.slider("Magnitude Range", 13.8, 32.61, (13.8, 32.61))
        diam_min = st.slider("Min Estimated Diameter (km)", 0.00, 4.62, (0.00, 4.62))
        diam_max = st.slider("Max Estimated Diameter (km)", 0.00, 10.33, (0.00, 10.33))

    with c2:
        velocity = st.slider("Relative Velocity", 1418.21, 173071.83, (1418.21, 173071.83))
        astro = st.slider("Astronomical Unit", 5.16453e-05, 0.4999515747)
        hazardous = st.selectbox("Potentially Hazardous?", options=[0, 1], index=0)

    with c3:
        start_date = st.date_input("Start Date", datetime(2024, 1, 1))
        end_date = st.date_input("End Date", datetime(2025, 4, 13))

    button = st.button("Apply Filter")

    if button:
        query = """
            SELECT
                asteroids.id, asteroids.name, asteroids.absolute_magnitude_h,
                asteroids.estimated_diameter_min_km, asteroids.estimated_diameter_max_km,
                asteroids.is_potentially_hazardous_asteroid,
                close_approach.neo_reference_id, close_approach.close_approach_date,
                close_approach.relative_velocity_kmph, close_approach.astronomical,
                close_approach.miss_distance_km,
                close_approach.orbiting_body
            FROM asteroids
            JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
            WHERE asteroids.absolute_magnitude_h BETWEEN %s AND %s
              AND asteroids.estimated_diameter_min_km BETWEEN %s AND %s
              AND asteroids.estimated_diameter_max_km BETWEEN %s AND %s
              AND close_approach.relative_velocity_kmph BETWEEN %s AND %s
              AND close_approach.close_approach_date BETWEEN %s AND %s
              AND asteroids.is_potentially_hazardous_asteroid = %s
        """
        params = [
            mag_min[0], mag_min[1],
            diam_min[0], diam_min[1],
            diam_max[0], diam_max[1],
            velocity[0], velocity[1],
            start_date, end_date,
            hazardous
        ]
        cursor.execute(query, params)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(rows, columns=columns)
        st.subheader(":satellite: Filtered Asteroids")
        st.dataframe(df, use_container_width=True)

# --- Queries Section ---
elif selected == "Queries":
    queries = {
        '1.Count how many times each asteroid has approached Earth': """
            SELECT asteroids.name, COUNT(*) AS approach_count
            FROM close_approach
            JOIN asteroids ON asteroids.id = close_approach.neo_reference_id
            GROUP BY asteroids.name
            ORDER BY approach_count DESC
        """,
        '2.Average velocity of each asteroid over multiple approaches': """
            SELECT asteroids.name, ROUND(AVG(close_approach.relative_velocity_kmph), 2) AS avg_velocity
            FROM close_approach
            JOIN asteroids ON asteroids.id = close_approach.neo_reference_id
            GROUP BY asteroids.name
            ORDER BY avg_velocity DESC
        """,
        '3.List top 10 fastest asteroids': """
            SELECT asteroids.name, close_approach.relative_velocity_kmph
            FROM close_approach
            JOIN asteroids ON asteroids.id = close_approach.neo_reference_id
            ORDER BY close_approach.relative_velocity_kmph DESC
            LIMIT 10
        """,
        '4.Find potentially hazardous asteroids that have approached Earth more than 3 times': """
            SELECT asteroids.name, COUNT(*) AS approach_count
            FROM close_approach
            JOIN asteroids ON asteroids.id = close_approach.neo_reference_id
            WHERE asteroids.is_potentially_hazardous_asteroid = 1
            GROUP BY asteroids.name
            HAVING approach_count > 3
            ORDER BY approach_count DESC
        """,
        '5.Find the month with the most asteroid approaches': """
            SELECT MONTH(close_approach_date) AS month, COUNT(*) AS total_approaches
            FROM close_approach
            GROUP BY month
            ORDER BY total_approaches DESC
            LIMIT 1
        """,
        '6.Get the asteroid with the fastest ever approach speed': """
            SELECT asteroids.name, close_approach.relative_velocity_kmph
            FROM close_approach
            JOIN asteroids ON asteroids.id = close_approach.neo_reference_id
            ORDER BY close_approach.relative_velocity_kmph DESC
            LIMIT 1
        """,
        '7.Sort asteroids by maximum estimated diameter (descending)': """
            SELECT name, estimated_diameter_max_km
            FROM asteroids
            ORDER BY estimated_diameter_max_km DESC
        """,
        '8.An asteroid whose closest approach is getting nearer over time': """
            SELECT asteroids.name, close_approach.close_approach_date, close_approach.miss_distance_km
            FROM close_approach
            JOIN asteroids ON asteroids.id = close_approach.neo_reference_id
            ORDER BY asteroids.name, close_approach.close_approach_date
        """,
        '9.Display the name of each asteroid along with the date and miss distance of its closest approach to Earth': """
            SELECT a.name, c.close_approach_date, c.miss_distance_km
            FROM close_approach c
            JOIN asteroids a ON a.id = c.neo_reference_id
            WHERE (a.name, c.miss_distance_km) IN (
                SELECT a2.name, MIN(c2.miss_distance_km)
                FROM close_approach c2
                JOIN asteroids a2 ON a2.id = c2.neo_reference_id
                GROUP BY a2.name
            )
        """,
        '10.List names of asteroids that approached Earth with velocity > 50,000 km/h': """
            SELECT DISTINCT asteroids.name, close_approach.relative_velocity_kmph
            FROM close_approach
            JOIN asteroids ON asteroids.id = close_approach.neo_reference_id
            WHERE close_approach.relative_velocity_kmph > 50000
        """,
        '11.Count how many approaches happened per month': """
            SELECT MONTH(close_approach_date) AS month, COUNT(*) AS count
            FROM close_approach
            GROUP BY month
            ORDER BY month
        """,
        '12.Find asteroid with the highest brightness (lowest magnitude value)': """
            SELECT name, absolute_magnitude_h
            FROM asteroids
            ORDER BY absolute_magnitude_h ASC
            LIMIT 1
        """,
        '13.Get number of hazardous vs non-hazardous asteroids': """
            SELECT is_potentially_hazardous_asteroid, COUNT(*) AS count
            FROM asteroids
            GROUP BY is_potentially_hazardous_asteroid
        """,
        '14.Find asteroids that passed closer than the Moon (less than 1 LD), with date and distance': """
            SELECT a.name, c.close_approach_date, c.miss_distance_lunar
            FROM close_approach c
            JOIN asteroids a ON a.id = c.neo_reference_id
            WHERE c.miss_distance_lunar < 1
        """,
        '15.Find asteroids that came within 0.05 AU': """
            SELECT a.name, c.close_approach_date, c.astronomical
            FROM close_approach c
            JOIN asteroids a ON a.id = c.neo_reference_id
            WHERE c.astronomical < 0.05
        """,
        '16.List the Most frequently orbiting body (other than Earth)': """
            SELECT orbiting_body, COUNT(*) AS count
            FROM close_approach
            WHERE orbiting_body != 'Earth'
            GROUP BY orbiting_body
            ORDER BY count DESC
            LIMIT 1
        """,
        '17.Calculate the Average miss distance by hazard type': """
            SELECT a.is_potentially_hazardous_asteroid, ROUND(AVG(c.miss_distance_km), 2) AS avg_miss_distance
            FROM close_approach c
            JOIN asteroids a ON a.id = c.neo_reference_id
            GROUP BY a.is_potentially_hazardous_asteroid
        """,
        '18.Find the Top 5 closest approaches to Earth': """
            SELECT a.name, c.close_approach_date, c.miss_distance_km
            FROM close_approach c
            JOIN asteroids a ON a.id = c.neo_reference_id
            ORDER BY c.miss_distance_km ASC
            LIMIT 5
        """,
        '19.Find the Total unique hazardous asteroids': """
            SELECT COUNT(DISTINCT name) AS total_hazardous
            FROM asteroids
            WHERE is_potentially_hazardous_asteroid = 1
        """,
        '20.Find the Asteroids that passed within 1 LD more than once': """
            SELECT a.name, COUNT(*) AS close_passes
            FROM close_approach c
            JOIN asteroids a ON a.id = c.neo_reference_id
            WHERE c.miss_distance_lunar < 1
            GROUP BY a.name
            HAVING close_passes > 1
        """
    }

    option = st.selectbox("Select your query", list(queries.keys()))

    # Execute and display selected query
    if option:
        cursor.execute(queries[option])
        result = cursor.fetchall()
        columns = [i[0] for i in cursor.description]
        df = pd.DataFrame(result, columns=columns)
        st.subheader(option)
        st.dataframe(df, use_container_width=True)
    else:
        st.warning("Query not implemented yet.")

Overwriting Nasa.py


In [ ]:
  !wget -q -O - ipv4.icanhazip.com  # Password will Generated

34.86.109.66


In [ ]:
!streamlit run Nasa.py & npx localtunnel --port 8501

⠙

⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.109.66:8501

⠴⠦⠧⠇⠏⠋⠙your url is: https://open-coats-flash.loca.lt
  Stopping...
^C
